In [1]:
from pathlib import Path
import os, sys
sys.path.append(str(Path(os.getcwd()).parent.parent))

### Creating a type and associated CLI

In [11]:
from params import Params, ConfStore
from dataclasses import dataclass
from params.params import cfg_from_cli

@dataclass
class MyType(Params):
    some_param: int
    some_other_param: str = 'lol'

cfg_from_cli accepts either a `schema`, a `base_config_name` or a `base_config`. Each argument will lead to different default values being used when parsing the CLI. 

`base_config_name` is a string that we look for in the ConfStore. The ConfStore is a global dictionary that holds configurations. *Be careful* not to inadvertently change someone's config in the ConfStore !  

In [12]:
sys.argv = "... --some_param 2".split()
print("From Schema", cfg_from_cli(schema=MyType))

base_config = MyType(some_param=3)
sys.argv = "... --some_other_param wow".split()
print("From BaseConfig", cfg_from_cli(base_config=base_config))

ConfStore["base"] = base_config
sys.argv = "... --some_other_param wow".split()
print("From BaseConfigName", cfg_from_cli(base_config_name="base"))

From Schema MyType(some_param=2, some_other_param='lol')
From BaseConfig MyType(some_param=3, some_other_param='wow')
From BaseConfigName MyType(some_param=3, some_other_param='wow')


We can also specify the whole config directly with the `--cfg` option

In [13]:
sys.argv = "... --cfg base".split()
print("From cfg", cfg_from_cli(schema=MyType))

From cfg MyType(some_param=3, some_other_param='lol')


## Nested Params
Params can be nested, in which case, the ConfStore can be used to shorten command line calls

In [16]:
@dataclass
class A(Params):
    value: int
        
@dataclass
class B(Params):
    value: int
    a: A
        
@dataclass
class C(Params):
    value: int
    b: B
        
sys.argv = "... --value 1 --b.value 2 --b.a.value 3".split()
print("From cli", cfg_from_cli(schema=C))

# now let's add things to the ConfStore
ConfStore["awesome_b"] = B(value=2, a=A(value=3))
sys.argv = "... --value 1 --b awesome_b".split()
print("Shorter cmd", cfg_from_cli(schema=C))

From cli C(value=1, b=B(value=2, a=A(value=3)))
Shorter cmd C(value=1, b=B(value=2, a=A(value=3)))


## A few utils

Params class can be exported as flat dictionaries or json

In [17]:
sys.argv = "... --value 1 --b awesome_b".split()
res = cfg_from_cli(schema=C)

print(res.to_json())

{
    "b": {
        "a": {
            "value": 3
        },
        "value": 2
    },
    "value": 1
}


Our config suck, so we have to mutate them. You can either use `\_\_post_init\_\_` to check that all arguments are correct, or you can implement a `_check_and_mutate_args`.
The latter will be called recursively on all fields of the class that are Params, allowing to do the checking and mutation of a Params class in one go.

In [19]:
@dataclass
class A(Params):
    value: int
    def _check_and_mutate_args(self):
        self.value += 1
        
@dataclass
class B(Params):
    value: int
    a: A
    def _check_and_mutate_args(self):
        self.value += 1
        
@dataclass
class C(Params):
    value: int
    b: B
    def _check_and_mutate_args(self):
        self.value += 1
        
sys.argv = "... --value 1 --b awesome_b".split()
res = cfg_from_cli(schema=C)
res.check_and_mutate_args()
print(res.to_json())  # everything is increased by 1 ! 

{
    "b": {
        "a": {
            "value": 4
        },
        "value": 3
    },
    "value": 2
}


## That's it.

You should know the main Params class we use is called `TrainerArgs` and is located in `evariste.trainer.args`